In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://seller-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)


IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential
Handle Credentials
Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: offer_received
Attributes: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


### Go to manufacturer notebook to send credential

In [4]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'initiator': 'external', 'role': 'holder', 'state': 'offer_received', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:Device_schema3:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:40:default', 'key_correctness_proof': {'c': '13029603041484820009090678822072701724751145947394037282597925237928789573163', 'xz_cap': '18300333132179616018659508919478391680140632015736620454617092977753532173134145181059450867531617670893741333381180969331537836965413802119968226024933461479497868177858240154182020731631789543307071622566683835461287286110328720536653318290672714627699769148319371271210999500691086273821835622131476517565067142868083179774066980545709048604756358693318247919022196155806179111176707071431011455831756592135078221282573876388032403302028064392213649968228958957457391517682180352378818412397543076562273286770807655026193325199544246298714735888074849156238367427489719587550729145883667184901231722698055592465802298154439805773366445672834551339754

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: offer_received
Being offered: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: request_sent
Attributes: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]
Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: request_sent
Handle Credentials
Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: credential_received
Attributes: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]


In [7]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: credential_received


### Seller store the credential in his wallet

In [8]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My Device Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: credential_acked
Attributes: [{'name': 'ProductionDate', 'value': '1672706006'}, {'name': 'ExpirationDate', 'value': '1704242006'}, {'name': 'Manufacturer', 'value': 'GOOD_COMPANY'}, {'name': 'Owner', 'value': 'GOOD_SELLER'}, {'name': 'Type', 'value': 'Dentistry'}, {'name': 'Status', 'value': '1'}]
Credential exchange a7267cc2-d107-42c5-b1b1-15e4b0ba984c, role: holder, state: credential_acked


In [9]:
response = await agent_controller.terminate()
